In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from amberdata_derivatives import AmberdataDerivatives
from matplotlib.ticker import MaxNLocator
import matplotlib.dates as mdates
import mplfinance as mpf
import numpy as np
import seaborn as sns
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
# insert API key below
amberdata_client = AmberdataDerivatives(api_key="")

amberdata_client.get

In [ ]:
# use this endpoint to get back all of the available instruments on deribit
instruments = pd.DataFrame(amberdata_client.get_instruments_information(exchange='deribit')['payload']['data'])
instruments.head(10)

In [ ]:
# plot put/call trade distribution
data = amberdata_client.get_trades_flow_block_volumes(exchange='deribit', currency='btc')
data = pd.DataFrame(data['payload']['data'])
data = data.groupby("putCall").sum()

plt.figure(figsize=(15, 7))
labels = ['Calls', 'Puts']
plt.pie(data.contractVolume.values.flatten(), labels=labels, autopct='%1.1f%%')
plt.title("BTC Put Call Trade Distribution Over the Past 7 Days")

In [ ]:
# plot block trade bar plot
data = amberdata_client.get_trades_flow_block_volumes(exchange='deribit', currency='btc', startDate='2024-06-14', endDate='2024-06-21')
data = pd.DataFrame(data['payload']['data'])

data['expirationTimestamp'] = pd.to_datetime(data.expirationTimestamp, unit='ms')
data['adj_volume'] = np.where(data.putCall=='P', -data.contractVolume, data.contractVolume)
# only look at expiries that are valid today 
data = data[data.expirationTimestamp >= pd.to_datetime(time.time(), unit='s')]

exps = set(data.expirationTimestamp)
store = []
for exp in exps:
    sub = data[data.expirationTimestamp == exp].reset_index(drop=True)    
    sub = sub[['expirationTimestamp', 'strike', 'putCall', 'adj_volume']]
    # filter out strikes to allow for better visual plotting
    sub = sub[(sub.strike > 60_000) & (sub.strike < 80_000)]
    sub['maturity'] = exp
    store.append(sub)
    
combined_df = pd.concat(store)
pivot_df = combined_df.pivot_table(index='strike', columns='maturity', values='adj_volume', aggfunc='sum').fillna(0)    

fig, ax = plt.subplots(figsize=(20, 10))
# Define colors for each maturity
colors = sns.color_palette("husl", len(pivot_df.columns))
maturities = pivot_df.columns

# Stack the bars
bottom = pd.Series([0]*len(pivot_df), index=pivot_df.index)
for maturity, color in zip(maturities, colors):
    sns.barplot(x=pivot_df.index, y=pivot_df[maturity], bottom=bottom, label=maturity, color=color)
    bottom += pivot_df[maturity]

ax.legend(title='Maturities')    
ax.set_xlabel('Strike Price')
ax.set_ylabel('Contract Volume')
yticks = ax.get_yticks()
yticklabels = ['C' + str(y) if y > 0 else 'P' + str(-y) if y < 0 else '0' for y in yticks]
ax.set_yticklabels(yticklabels);
plt.title("BTC Block Volume Traded: 2024-06-14 to 2024-06-21");

In [ ]:
# plot top 10 instruments traded
data = amberdata_client.get_instruments_most_traded(exchange='deribit', currency='BTC', startDate='2024-06-07', endDate='2024-06-12')
data = pd.DataFrame(data['payload']['data'])

plt.figure(figsize=(15, 7))
plt.barh(data.instrument[:10], data.contractVolume[:10][::-1])
plt.title("BTC Top 10 Instruments Traded: 2024-06-07 to 2024-06-12")
plt.xlabel("Number of Contracts Traded");

In [ ]:
# plot trade distribution
data = amberdata_client.get_trades_flow_put_call_distribution(exchange='deribit', currency='BTC', startDate='2024-06-07', endDate='2024-06-12')
data = pd.DataFrame(data['payload']['data'])

plt.figure(figsize=(15, 7))
labels = ['Calls Bought', 'Puts Bought', 'Calls Sold', 'Puts Sold']
plt.pie(data.iloc[:, :4].values.flatten(), labels=labels, autopct='%1.1f%%')
plt.title("BTC Put Call Trade Distribution: 2024-06-07 to 2024-06-12");

In [ ]:
# plot normalized gamma USD
data = amberdata_client.get_trades_flow_gamma_exposures_normalized_usd(exchange='deribit', currency='btc')
data = pd.DataFrame(data['payload']['data'])
data.index = pd.to_datetime(data.snapshotTimestamp, unit='ms')

plt.figure(figsize=(15, 7))
plt.plot(data.normalizedGammaUSD/1_000_000)
plt.axhline(0, linestyle='--', c='r')
plt.title("BTC Normalized Gamma Exposure USD: 2022-11-17 to 2024-06-13")
plt.ylabel("USD Gamma Exposure ($ Millions)");

In [ ]:
# table of gex snapshots across different instruments
data = amberdata_client.get_trades_flow_gamma_exposures_snapshots(exchange='deribit', currency='btc', startDate='2024-01-01', endDate='2024-01-02')
data = pd.DataFrame(data['payload']['data'])
data

In [ ]:
# plot top 10 largest vega trades
data = amberdata_client.get_options_scanner_top_trades(exchange='deribit', currency='btc', startDate='2024-06-07', endDate='2024-06-12')
data = pd.DataFrame(data['payload']['data'])
# only look at top 10 
data = data.sort_values("sizeVega", ascending=False).head(10).reset_index(drop=True)
# sort by descending order 
plt.figure(figsize=(15, 7))
plt.barh(data.instrument, data.sizeVega[::-1])
plt.title("BTC Largest 10 Vega Option Trades: 2024-06-07 to 2024-06-12")
plt.xlabel("Size Vega")